# Active learning to augment racism dataset

In [1]:
# import libraries 
import pandas as pd
# pd take screen width
pd.set_option('display.max_colwidth', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import string
%matplotlib inline


In [2]:
arabic_stopwords = set(nltk.corpus.stopwords.words("arabic"))

arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations = arabic_punctuations + english_punctuations


def remove_urls (text):
    text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE)
    return text


def remove_emails(text):
    text = re.sub(r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)", "",  text, flags=re.MULTILINE)
    return text

# def remove_emoji(text):
#     return emoji.get_emoji_regexp().sub(u'', text)

def remove_emoji(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

def normalization(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text

def remove_stopwords(text):
    filtered_sentence = [w for w in text.split() if not w in arabic_stopwords]
    return ' '.join(filtered_sentence)

def cleaning_content(line):
    if (isinstance(line, float)):
        return None
    line.replace('\n', ' ')
    line = remove_emails(line)
    line = remove_urls(line)
    line = remove_emoji(line)
    nline = [w if '@' not in w else 'USERID' for w in line.split()]
    line = ' '.join(nline)
    line = line.replace('RT', '').replace('<LF>', '').replace('<br />','').replace('&quot;', '').replace('<url>', '').replace('USERID', '')


    # add spaces between punc,
    line = line.translate(str.maketrans({key: " {0} ".format(key) for key in punctuations}))

    # then remove punc,
    translator = str.maketrans('', '', punctuations)
    line = line.translate(translator)

    line = remove_stopwords(line)
    line=remove_diacritics(normalization(line))
    return line

def hasDigits(s):
    return any( 48 <= ord(char) <= 57  or 1632 <= ord(char) <= 1641 for char in s)


In [3]:
# read the data
df = pd.read_csv('../all.csv')
pool = pd.read_csv('../DataSet/final_dataset.csv', encoding='utf-16')

print(df.shape)
df.head()

(2133, 4)


,id,text,Majority_Label,label
0,1,البطل قاتل وجاذف بحياته لتحيا انت واطي عيب الشوم,1.0,Verbal abuse
1,2,انو غريب يوجعك راسك القواتجيه عاده بيكونو بلا راس,1.0,Racism
2,3,طاءفي روح اسال اهل قانا وهني بخبروك,1.0,Racism
3,4,غرد الجحش وظن باءنه حصانا تتكلم العوني بموضوعيه فيرد كبهيم حقير جربان اصله,1.0,Verbal abuse
4,5,الفلسطينيين واللبنانين والشاميين لكان انتو لليوم تشربوا بول البعير تاكلون بصحن وتبزقون منو,1.0,Racism


In [4]:
print(pool.shape)
pool.head()

(31757, 2)


,Comment,Majority_Label
0,وزير الخارجيه اللبناني جبران باسيل قال سلسله تغريداته عقب اختتام القمه العربيه التنمويه الاقتصاديه والاجتماعيه لمسنا تج,0.0
1,سوريه بلد الحضارات تربطها بعليه او بحيوان,0.0
2,اخي الحاج اذا شعرت انك محرجا الانتقادات لتصريح جبران باسيل داعي لان تهاجم المنتقدين,0.0
3,فيك تعيش بلا تكب فتن ليل نهار وبكره قلهم الموضوع السيد,0.0
4,البطل قاتل وجاذف بحياته لتحيا انت واطي عيب الشوم,1.0


In [5]:
# drop nan values
df.dropna(inplace=True)
pool.dropna(inplace=True)
print( f" le df : {len(df)} , len pool {len(pool)} ")

 le df : 2125 , len pool 31755 


In [6]:
# substract the data from the pool
pool = pool[~pool['Comment'].isin(df['text'])]

print(len(pool))

28895


In [7]:
# see nan values
print(pool.isnull().sum())

# drop nan values
pool.dropna(inplace=True)

# clean the data
pool['Comment'] = pool['Comment'].apply(cleaning_content)

Comment           0
Majority_Label    0
dtype: int64


In [8]:
# select label racism
racism = df[df['label'] == 'Racism']

# delete label racism from df
df = df[df['label'] != 'Racism']

# concat racism and df
df2 = pd.concat([df, racism])

# rename label diffrent than racism to 'not racism'
df2['label'] = df2['label'].apply(lambda x: 'not racism' if x != 'Racism' else x)

# shuffle the data
df2 = df2.sample(frac=1).reset_index(drop=True)

# balance the data
df2 = df2.groupby('label').apply(lambda x: x.sample(145, replace=True)).reset_index(drop=True)


In [9]:
# clean the data
df2['text'] = df2['text'].apply(cleaning_content)

print(df2.label.value_counts())

df2.head()

Racism        145
not racism    145
Name: label, dtype: int64


,id,text,Majority_Label,label
0,13597,الي جهنم وبءس المصير يطلق عليهم المثقفون العرب كانوا داءما مناصرين للطغاه,1.0,Racism
1,12874,والله كنعرفك بنت داركم مايلوحك انت عرفك بغيت خربي عاءله باش تبني عاءلتك كنت العروبيه ديال سيدي حجاج اشداك لشي محامي معاريف طوموبيلات,1.0,Racism
2,13239,انت تريد احد تنظم الدوله الجزيره نسمح حشد الفرس المجوسي الشيعي ان يبقي احد منكم بلادنا الله تتكلم وكان الجزيره ملك لابوك,1.0,Racism
3,13389,شوف هاد شلح تفو فين توصل لمره الحامض,1.0,Racism
4,13076,اغلب الناس تتحدث السيسي وتتناسي السيسي الا دميه يحركها النظام السعودي النظام السعودي الا الحبل يحرك الدميه والحبل تحركه امريكا,1.0,Racism


In [10]:
len(df2)

290

In [11]:
# tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# split the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df2['text'], df2['label'], test_size=0.2, random_state=42)

# vectorize the data
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# train the model
from sklearn.naive_bayes import MultinomialNB

MNB_model = MultinomialNB().fit(X_train, y_train)

# test the model
predicted_mnb = MNB_model.predict(X_test)

# evaluate the model
from sklearn import metrics
print(metrics.classification_report(y_test, predicted_mnb))


              precision    recall  f1-score   support

      Racism       0.68      0.89      0.77        28
  not racism       0.86      0.60      0.71        30

    accuracy                           0.74        58
   macro avg       0.77      0.75      0.74        58
weighted avg       0.77      0.74      0.74        58



In [12]:
# print probabilities
probs_mnb = MNB_model.predict_proba(X_test)

for i in range(len(probs_mnb)):
    print(predicted_mnb[i], probs_mnb[i],y_test.iloc[i])

Racism [0.81316828 0.18683172] Racism
Racism [0.59980612 0.40019388] not racism
Racism [0.83309096 0.16690904] Racism
not racism [0.44852296 0.55147704] not racism
Racism [0.53498495 0.46501505] Racism
not racism [0.47017659 0.52982341] Racism
not racism [0.27618952 0.72381048] not racism
Racism [0.6001534 0.3998466] Racism
Racism [0.76600121 0.23399879] Racism
Racism [0.6843156 0.3156844] Racism
Racism [0.62394813 0.37605187] not racism
not racism [0.35764774 0.64235226] not racism
Racism [0.53504004 0.46495996] Racism
not racism [0.32273232 0.67726768] not racism
Racism [0.55665261 0.44334739] not racism
Racism [0.58934682 0.41065318] Racism
Racism [0.83309096 0.16690904] Racism
Racism [0.81387075 0.18612925] Racism
not racism [0.41375901 0.58624099] not racism
not racism [0.41685557 0.58314443] not racism
Racism [0.76424945 0.23575055] Racism
not racism [0.35469276 0.64530724] not racism
not racism [0.33667428 0.66332572] not racism
Racism [0.70014946 0.29985054] Racism
Racism [0.88

In [13]:
# Trying svm
from sklearn import svm
svm_model = svm.SVC(kernel='linear', C=1, probability=True).fit(X_train, y_train)
predicted_svm = svm_model.predict(X_test)
print(metrics.classification_report(y_test, predicted_svm))

# print probabilities
probs_svm = svm_model.predict_proba(X_test)

for i in range(len(probs_svm)):
    print(predicted_svm[i], probs_svm[i], y_test.iloc[i])

              precision    recall  f1-score   support

      Racism       0.69      0.71      0.70        28
  not racism       0.72      0.70      0.71        30

    accuracy                           0.71        58
   macro avg       0.71      0.71      0.71        58
weighted avg       0.71      0.71      0.71        58

Racism [0.97450733 0.02549267] Racism
Racism [0.65157806 0.34842194] not racism
Racism [0.97456048 0.02543952] Racism
not racism [0.23143678 0.76856322] not racism
not racism [0.436879 0.563121] Racism
not racism [0.3202784 0.6797216] Racism
not racism [0.01507297 0.98492703] not racism
Racism [0.8726249 0.1273751] Racism
Racism [0.97453481 0.02546519] Racism
Racism [0.92760192 0.07239808] Racism
Racism [0.61006147 0.38993853] not racism
not racism [0.07326671 0.92673329] not racism
not racism [0.44338263 0.55661737] Racism
not racism [0.04303691 0.95696309] not racism
not racism [0.52663443 0.47336557] not racism
Racism [0.75899953 0.24100047] Racism
Racism [0.974

In [14]:
# drop column label
pool.head()

pool = pool[pool['Majority_Label'] == 1]

In [15]:
pool.head()

,Comment,Majority_Label
55,اودع مليار عشان حزب الشيطان بلبنان يمر بمرحله صعبه توقف الدعم الايران اثر الحصار عليها,1.0
1041,انت بوق ايراني مجوسي رخيص سعرك كلفول السوداني,1.0
1102,خوش معارض واطي والله,1.0
1150,تعلق الشعب الخري بطريقه الشعب الخري,1.0
1166,جد جحاش القوات اذار نهيق نهيق,1.0


In [16]:
# predict the df
predicted_pool_MNB = MNB_model.predict(vectorizer.transform(pool['Comment']))
predicted_pool_prob_MNB = MNB_model.predict_proba(vectorizer.transform(pool['Comment']))

# add the prediction to the pool
pool['prediction_MNB'] = predicted_pool_MNB
# take the max probability
pool['probability_MNB'] = [max(x) for x in predicted_pool_prob_MNB]

# predict the df
predicted_pool_svm = svm_model.predict(vectorizer.transform(pool['Comment']))
predicted_pool_prob_svm = svm_model.predict_proba(vectorizer.transform(pool['Comment']))

# add the prediction to the pool
pool['prediction_svm'] = predicted_pool_svm
# take the max probability
pool['probability_svm'] = [max(x) for x in predicted_pool_prob_svm]


pool.head()

,Comment,Majority_Label,prediction_MNB,probability_MNB,prediction_svm,probability_svm
55,اودع مليار عشان حزب الشيطان بلبنان يمر بمرحله صعبه توقف الدعم الايران اثر الحصار عليها,1.0,not racism,0.632642,not racism,0.904646
1041,انت بوق ايراني مجوسي رخيص سعرك كلفول السوداني,1.0,Racism,0.600151,not racism,0.535087
1102,خوش معارض واطي والله,1.0,not racism,0.544855,not racism,0.727424
1150,تعلق الشعب الخري بطريقه الشعب الخري,1.0,Racism,0.519762,not racism,0.500000
1166,جد جحاش القوات اذار نهيق نهيق,1.0,Racism,0.504310,not racism,0.624900


In [17]:
# get the racist comments with probability > 0.8
racist_comments = pool[pool['prediction_MNB'] == 'Racism']
racist_comments = racist_comments[racist_comments['probability_MNB'] > 0.75]
racist_comments = racist_comments[racist_comments['probability_svm'] > 0.85]

print(len(racist_comments))
# add column to racist comments called final_label
racist_comments['final_label'] = 'Nan'

racist_comments

23


,Comment,Majority_Label,prediction_MNB,probability_MNB,prediction_svm,probability_svm,final_label
7863,ديما الواطيه اوطي سرمايتي هيدا احلي,1.0,Racism,0.751859,Racism,0.860425,Nan
8145,عمي ديما صادق بدها تنشهر صدقوني انتشر هاشتاغ الواطيه ديما ديما كتير كانت مبسوطه انو تريند,1.0,Racism,0.759668,Racism,0.935775,Nan
8158,ديما الوطيه هيدا انتصار غشيمه فلننتظر,1.0,Racism,0.751859,Racism,0.860425,Nan
9477,عم تمزحي مش انو مزحك تقيل الا انو كذابه كمان,1.0,Racism,0.754203,Racism,0.881441,Nan
10238,انو شبعتي كلي خرااااا,1.0,Racism,0.755131,Racism,0.875984,Nan
10739,ديما صادق تاكدي انو كمان انتي رح فيكي تركبي موجه الثوره شو عملتي خليكي عم تقدمي نشرات ال,1.0,Racism,0.750811,Racism,0.917705,Nan
15986,الاهم انو اعطانا حقاره ولءم وخساسه وجع يخلع نيعك,1.0,Racism,0.755131,Racism,0.875984,Nan
19277,صح الخبر فخيرا فعلوا دابوا طعن العرب والمسلمين ظهورهم حقدهم القومي والعنصري حدود مكان لابناء اسراءيل الثانيه ارض العرب والمسلمين,1.0,Racism,0.776974,Racism,0.987650,Nan
19387,علاقه ايران اسراءيل وليس الا بروسيا فلتاكل الطحين شعوب العملاء وبما تتغدي يوم بينما اارضهم اغني الارض,1.0,Racism,0.753408,Racism,0.983002,Nan
19885,السيسي وعباس كبار الصهاينه العرب وعملاء اسراءيل,1.0,Racism,0.776110,Racism,0.986160,Nan


In [22]:
# manually check if the comments are racist or not by prompting the user to enter 1 or 0
for i in range(len(racist_comments)):
    print(f'text = {racist_comments.iloc[i]["Comment"]} , MNB proba = {racist_comments.iloc[i]["probability_MNB"]} , svm proba = {racist_comments.iloc[i]["probability_svm"]}')
    print('---------------------------------------')
    racist_comments.iloc[i, racist_comments.columns.get_loc('final_label')] = input('enter 1 if the comment is racist, 0 otherwise : ')?1 'Racism' : 'not racism'
    print('---------------------------------------')

text = ديما الواطيه اوطي سرمايتي هيدا احلي , MNB proba = 0.7518588208918577 , svm proba = 0.8604251155221339
---------------------------------------
---------------------------------------
text = عمي ديما صادق بدها تنشهر صدقوني انتشر هاشتاغ الواطيه ديما ديما كتير كانت مبسوطه انو تريند , MNB proba = 0.7596682931144175 , svm proba = 0.9357747118784137
---------------------------------------
---------------------------------------
text = ديما الوطيه هيدا انتصار غشيمه فلننتظر , MNB proba = 0.7518588208918577 , svm proba = 0.8604251155221339
---------------------------------------
---------------------------------------


In [23]:
racist_comments

,Comment,Majority_Label,prediction_MNB,probability_MNB,prediction_svm,probability_svm,final_label
7863,ديما الواطيه اوطي سرمايتي هيدا احلي,1.0,Racism,0.751859,Racism,0.860425,2
8145,عمي ديما صادق بدها تنشهر صدقوني انتشر هاشتاغ الواطيه ديما ديما كتير كانت مبسوطه انو تريند,1.0,Racism,0.759668,Racism,0.935775,2
8158,ديما الوطيه هيدا انتصار غشيمه فلننتظر,1.0,Racism,0.751859,Racism,0.860425,2
9477,عم تمزحي مش انو مزحك تقيل الا انو كذابه كمان,1.0,Racism,0.754203,Racism,0.881441,Nan
10238,انو شبعتي كلي خرااااا,1.0,Racism,0.755131,Racism,0.875984,Nan
10739,ديما صادق تاكدي انو كمان انتي رح فيكي تركبي موجه الثوره شو عملتي خليكي عم تقدمي نشرات ال,1.0,Racism,0.750811,Racism,0.917705,Nan
15986,الاهم انو اعطانا حقاره ولءم وخساسه وجع يخلع نيعك,1.0,Racism,0.755131,Racism,0.875984,Nan
19277,صح الخبر فخيرا فعلوا دابوا طعن العرب والمسلمين ظهورهم حقدهم القومي والعنصري حدود مكان لابناء اسراءيل الثانيه ارض العرب والمسلمين,1.0,Racism,0.776974,Racism,0.987650,Nan
19387,علاقه ايران اسراءيل وليس الا بروسيا فلتاكل الطحين شعوب العملاء وبما تتغدي يوم بينما اارضهم اغني الارض,1.0,Racism,0.753408,Racism,0.983002,Nan
19885,السيسي وعباس كبار الصهاينه العرب وعملاء اسراءيل,1.0,Racism,0.776110,Racism,0.986160,Nan


In [123]:
sheikh = pd.read_csv('../train_tweets.csv', encoding='utf-16')
sheikh_test = pd.read_csv('../test_tweets.csv', encoding='utf-16')

sheikh.head()

,id,tweet,hate
0,930187100225404928,بمجرد ما ان يفرضوا عليك الاسلام تتحول حياتك الي الجحيم و تبقي تتساءل ما هو الحرام و ما هو حلال و هل هذا جاءز ام ذنب ام سيءه ام حسنه و من هي خير امه و من هي شر امه و من هم المءمنون و من هم الكافرون و اليهود الذين لعنهم الله و رسوله و دعي المسلمين لقتالهم,1
1,930188185824583680,مش معقول مفتي السعوديه اكبر دوله سنيه يفتي بمسانده اليهود والوزير الصهيوني يدعوه للزياره لطفك بنا يارب,1
2,930188285065953281,انا اولكم وباذن الله كل التوفيق ل اوراوا الياباني علي النادي اليهودي القذر السبت كلنا ضد الهلال,1
3,930188474438778886,تيار المحبه اليمن رءيس الهيءه العليا تيار المحبه اليمن علي اليماني يطلق مبادره المحبه التاريخيه بين العرب اليهود فلسطين المحتله مقابل العوده الي حدود ثوره ما قبل ثوره 48 earthquake ITASWE Russia2018 earthquake Pakistan MotoGP,0
4,930188524321693702,قالها الاكسبشني عنخيركم ما تنامون عنخيركم شنو يهود انتوا يهود,1


In [130]:
# balance the data
sheikh = sheikh.groupby('hate').apply(lambda x: x.sample(1305, replace=True)).reset_index(drop=True)

sheikh.hate.value_counts()

0    1305
1    1305
Name: hate, dtype: int64

In [124]:
sheikh_test.head()

,id,tweet,hate
0,958390144494694400,اخطر اعداء الامه منافقوها ينشغلون بتشويه مصلحيها ليضعفوهم ويسكتون عن مفسديها ليقووهم كانوا يسخرون من الصحابه ويسكتون عن يهود المدينه فلاجل ذلك حاربوا الصحوه لاجل هذا حاربوا الصحوه,1
1,958390531867971584,حتي مارك اليهودي يحب مخرجات مءتمر الخوار الوطني قبل قليل ارسل لي رساله تم اعاده تغيير صورتك الشخصيه الي الصوره,0
2,958390679780016129,gcc تصوير جوي لحصن مرحب في خيبر الذي فتحه النبي صلي الله عليه وسلم في معركه خيبر وكان مقر اليهود بعد اجلاءهم من المدينه المنوره,0
3,958390811514822656,خدمات الكنيسه ماشيين مع بعض بمبدا اليهود لا يعاملون السامريين,0
4,958390828682096640,ذكر الله المنافقين اكثر من اليهود لانهم يستعملون وساءل شرعيه لهدم الاصول ويخفون علي العامه فاعين البسطاء علي الوساءل واعينهم علي الاهداف ف لاجل هذا حاربوا الصحوه,0


In [131]:
tfidf_train = vectorizer.transform(sheikh['tweet'])
y_sheikh = sheikh['hate']

tfidf_test = vectorizer.transform(sheikh_test['tweet'])
y_sheikh_test = sheikh_test['hate']

svm_model2 = svm.SVC(kernel='linear', C=1, probability=True).fit(tfidf_train, y_sheikh)

# test the model
predicted_svm2 = svm_model2.predict(tfidf_test)

# evaluate the model
from sklearn import metrics
print(metrics.classification_report(y_sheikh_test, predicted_svm2))

# confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_sheikh_test, predicted_svm2)


              precision    recall  f1-score   support

           0       0.70      0.63      0.66       180
           1       0.51      0.58      0.54       117

    accuracy                           0.61       297
   macro avg       0.60      0.61      0.60       297
weighted avg       0.62      0.61      0.62       297



array([[114,  66],
       [ 49,  68]], dtype=int64)

In [132]:
MNB_model2 = MultinomialNB().fit(tfidf_train, y_sheikh)

# test the model
predicted_mnb2 = MNB_model2.predict(tfidf_test)

# evaluate the model
from sklearn import metrics
print(metrics.classification_report(y_sheikh_test, predicted_mnb2))

# confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_sheikh_test, predicted_mnb2)


              precision    recall  f1-score   support

           0       0.66      0.63      0.64       180
           1       0.46      0.49      0.48       117

    accuracy                           0.58       297
   macro avg       0.56      0.56      0.56       297
weighted avg       0.58      0.58      0.58       297



array([[114,  66],
       [ 60,  57]], dtype=int64)